In [11]:
⎕PW←5000

## Part one

This problem is about detecting a sequence of characters in a larger string. One way to approach that is to look at characters in a sliding window. Luckily, we can get sliding windows through an operator we've seen before: reduce (/)!

In [20]:
⊢input←'mjqjpqmgbljsphdztnvjfqwrcgsmlb'
4,/input

mjqjpqmgbljsphdztnvjfqwrcgsmlb

┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│mjqj│jqjp│qjpq│jpqm│pqmg│qmgb│mgbl│gblj│bljs│ljsp│jsph│sphd│phdz│hdzt│dztn│ztnv│tnvj│nvjf│vjfq│jfqw│fqwr│qwrc│wrcg│rcgs│cgsm│gsml│smlb│
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘

Let's break down what's happening here a little more. As we've seen before, reduce takes a function on the left and an array on the right, then it reduces the array using the operator:

In [15]:
+/1 2 3
1+2+3

6

6

Here, our function is "concatenate" (,), which joins arrays:

In [17]:
1 2,3 4

1 2 3 4

The final piece is that "4". How can reduce take it as a left argument when it's already taking concatenate? Well, it's time to finally examine the full APL operator specification.

At most, a symbol can take right and left *function* arguments *and* right and left *data* arguments. If `x` is our operator, it looks like this:
```
⍺ ⍺⍺ x ⍵⍵ ⍵
```
where ⍺⍺ and ⍵⍵ are functions, and ⍺ and ⍵ are data.

A good way to show this off is with a dfn. As you'd expect, it can access all four arguments:

In [57]:
x←{(⍺⍺/⍺),⍵⍵/⍵}
1 2 3 +x- 4 3 2

6 3

While that dfn isn't useful per se, it's a good demonstration of how APL parses symbols. Most symbols don't accept all four arguments, but knowing how the parser reads symbols is useful.

Returning to our problem, we now have a sliding window across the input:

In [58]:
4,/input

┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│mjqj│jqjp│qjpq│jpqm│pqmg│qmgb│mgbl│gblj│bljs│ljsp│jsph│sphd│phdz│hdzt│dztn│ztnv│tnvj│nvjf│vjfq│jfqw│fqwr│qwrc│wrcg│rcgs│cgsm│gsml│smlb│
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘

Next we need to look for duplicate letters in these. For that, we can use "unique" (monadic ∪), which removes duplicate elements from an array:

In [59]:
∪¨4,/input

┌───┬───┬───┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│mjq│jqp│qjp│jpqm│pqmg│qmgb│mgbl│gblj│bljs│ljsp│jsph│sphd│phdz│hdzt│dztn│ztnv│tnvj│nvjf│vjfq│jfqw│fqwr│qwrc│wrcg│rcgs│cgsm│gsml│smlb│
└───┴───┴───┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘

We can simplify this further by simply reducing with ∪. When used dyadically, it treats its left and right arguments like sets and unions them, skipping duplicates.

In [68]:
1 2 3∪3 4 5

1 2 3 4 5

In [66]:
4∪/input

┌───┬───┬───┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│mjq│jqp│qjp│jpqm│pqmg│qmgb│mgbl│gblj│bljs│ljsp│jsph│sphd│phdz│hdzt│dztn│ztnv│tnvj│nvjf│vjfq│jfqw│fqwr│qwrc│wrcg│rcgs│cgsm│gsml│smlb│
└───┴───┴───┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘

Now we just need the first window that still has four elements:

In [70]:
≢¨4∪/input

3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4

To find that element, we can use "index of" (dyadic ⍳). It takes a thing to look for on the right and an array to search in on the left, and returns the first index where it finds that thing (remember that APL starts array indexes at 1). Since our search array is already on the right, it's convenient to flip (⍨) ⍳.

In [73]:
1 2 3⍳3
4⍳⍨≢¨4∪/input

3

4

The first window contains characters 1-4, the second 2-5, the third 3-6, etc. Generally, to get the index (starting from 1) of the last character in a window, you do `(window size - 1) + index`.

In [129]:
3+4⍳⍨≢¨4∪/input

7

## Part 2
The only difference between parts one and two is the window size. Let's take this opportunity to factor the window size out and turn the solution into a dfn:

In [78]:
sol←{⍺+1-⍨⍺⍳⍨≢¨⍺∪/⍵}
4 sol input
14 sol input

7

19

We could even do both at the same time and try all the examples simultaneously! Note that when one of the arguments of each (¨) is either a scalar or a length-1 array, it reuses it across all the inputs of the other argument. That's why we have to enclose (⊂) the input. (That's *rank-polymorphism* again!)

In [86]:
4 14sol¨⊂input

7 19

We could even go a step further and test all of the test strings using an outer product!

In [87]:
4 14 ∘.sol 'mjqjpqmgbljsphdztnvjfqwrcgsmlb' 'bvwbjplbgvbhsrlpgdmjqwftvncz' 'nppdvjthqldpwncqszvftbrmjlhg' 'nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg' 'zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw'

7 5 6 10 11
19 23 23 29 26

## Digressions
Today I also wanted to write a bit about some other ways of solving parts of this problem. The one provided above is certainly the most compact option, but there are some interesting ideas to examine in other approaches.

### Identifying the start marker
Above, we used `⍺=≢¨` to find the first window with all unique characters. Another way to do that would've been to unique the window and then compare it to itself. In that case, rather than `4∪/⍵`, we'd want to preserve the original input and then ¨ over it.

In [88]:
4,/input
∪¨4,/input

┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│mjqj│jqjp│qjpq│jpqm│pqmg│qmgb│mgbl│gblj│bljs│ljsp│jsph│sphd│phdz│hdzt│dztn│ztnv│tnvj│nvjf│vjfq│jfqw│fqwr│qwrc│wrcg│rcgs│cgsm│gsml│smlb│
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘

┌───┬───┬───┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│mjq│jqp│qjp│jpqm│pqmg│qmgb│mgbl│gblj│bljs│ljsp│jsph│sphd│phdz│hdzt│dztn│ztnv│tnvj│nvjf│vjfq│jfqw│fqwr│qwrc│wrcg│rcgs│cgsm│gsml│smlb│
└───┴───┴───┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘

Then we can use a fork to unique each window and compare it to itself: Unlike = which compares arrays element-wise, ≡ (match) compares arrays in their entirety.

In [95]:
1 2 3 = 1 2 2
1 2 3 ≡ 1 2 3
(∪≡⊢)¨4,/input

1 1 0

1

0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

Then like before, we could search for the first 1:

In [94]:
1⍳⍨(∪≡⊢)¨4,/input

4

### Comparing arrays
Let's imagine for a moment that ≡ didn't exist and we wanted to keep the above method for finding the start marker. In that case we could use ≢ (tally) and compare the array lengths instead:

In [99]:
(≢∘∪=≢)¨4,/input

0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

Let's take a closer look at the pattern in that fork. We're applying the same function to both sides, and then combining them with another function. Essentially:
```
(g y) f (g x)
```
That's different from a fork, which is:
```
(f x) h (g x)
```
But it feels like there should be some way to factor this pattern out, right? Well, that's what ⍥ (over) is for! It's useful when you want to pass arguments through some kind of transformation before operating on them. Here's its full definition:
```
X f⍥g Y = (g x)f(g Y)
```
For example:

In [103]:
1 2 3,⍥≢3 2 1

3 3

In our case, we can combine ⍥ with a fork to unique the right, tally both, and compare equality. Here, `=⍥≢` is acting as the combining middle function in the fork.

In [104]:
(≢∘∪=≢)¨4,/input
(∪ =⍥≢ ⊢)¨4,/input

0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

That's definitely a little hard to read the first time you look at it, so it might be helpful to look at how it's parsed:

In [105]:
(≢∘∪=≢)
(∪ =⍥≢ ⊢)

┌─┼─┐
 ∘ = ≢
┌┴┐
≢ ∪

┌─┼─┐
∪ ⍥ ⊢
 ┌┴┐
 = ≢

Here's how the data flows through the code:
```
      ┌►∪──►≢─┐
      │       ▼
input─┤       =─►
      │       ▲
      └►⊢──►≢─┘
```

### Finding the index of the first matching window

Let's say we're using one of the implementations that produces a binary array. For example:

In [117]:
input2←'nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg'
(∪≡⊢)¨4,/input2

0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In that case, there's another way to find the first matching window. Notice that if we negate the array, we get a contiguous stretch of 1s until the window is found. If you add that chunk of 1s up, you'll get its index - 4 (because there are 4 items in the first window)

In [120]:
4,/input2
~(∪≡⊢)¨4,/input2

┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│nznr│znrn│nrnf│rnfr│nfrf│frfn│rfnt│fntj│ntjf│tjfm│jfmv│fmvf│mvfw│vfwm│fwmz│wmzd│mzdf│zdfj│dfjl│fjlv│jlvt│lvtq│vtqn│tqnb│qnbh│nbhc│bhcp│hcpr│cprs│prsg│
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘

1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

If we can split the array up on 0s and take the first item, we'll have that stretch of 1s and can add it up. Partition (⊆) is the tool for the job again. One interesting thing this time is that we want to partition this array on itself! For that, we can use self/swap (⍨). When it's not given a left argument, it repeats the right argument on the left:

In [126]:
a←~(∪≡⊢)¨4,/input2
a⊆a
⍝ Is the same as
⊆⍨a

┌───────────┬─┐
│1 1 1 1 1 1│1│
└───────────┴─┘

┌───────────┬─┐
│1 1 1 1 1 1│1│
└───────────┴─┘

Now we can grab the first one with "first" (⊃) and add-reduce, and we've got our solution.

In [128]:
4++/⊃⊆⍨~(∪≡⊢)¨4,/input2

10